<a href="https://colab.research.google.com/github/PodevynLoris/Mobile_robot_simulator/blob/main/07_1_ANLP_Word_Embeddings_2025_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Start by copying this into your Google Drive!!

Maastricht_University_logo.svg

# Advanced Natural Language Processing Course - Tutorial Word Embeddings
Author: Jan Scholtes en Gijs Wijngaard






---

Version 2025-2026.1

---

Welcome to the tutorial about Word Embeddings, a key component of Deep-Learning NLP applications. In this notebook you will work with Word2Vec models, and we finish off with transformers and sentence transformers.

We will also discuss several limitations of Word Embeddings. Like tokenization errors, wrong use of word embeddings will propagate through the entire NLP pipeline and negatively influence the performance of your model.

There are many sources of possible errors:
- Wrong language (do not overestimate multi-language claims)
- Wrong character set (missing characters)
- Wrong tokenization leading to unknown or corrupted tokens for which no embeddings exist or that cannot be dealt with sub-words
- No relevant context for sub-word embeddings
- Missing embeddings for chemical formulas, DNA/RNA, abbreviations, etc  
- Missing embeddings in general (out of vocabulairy errors) and limited sub-words
- Bias in embeddings
- ....

One can state  that there are many ways to do this wrong and a few ways to do it right! In this tutorial we will look at a few of these problems and how to address them.


---

## Word2Vec
In this section, we will focus on word representations. We will train a Word2Vec model from scratch, by using the same dataset as before. In this way, we try to compare the two datasets and see if we can find differences between words in a negative setting vs words in a positive setting.

Word2Vec learns its word embeddings by looking inside the documents and checking the nearby words. The core idea behind this is that similar words are nearby in a sentence.
The most common implementation for Word2Vec in Python is the one by [gensim](https://radimrehurek.com/gensim/models/word2vec.html). We can compute the embeddings by passing our documents as sentences to the model. Then to get an embedding, we just index the models word vectors with our needed embedding:

In [ ]:
import nltk
nltk.download('movie_reviews')
nltk.download('words')
from nltk.corpus import words, movie_reviews as mr
nltk_words = set(words.words())
import string
from collections import Counter
def remove_punct(word):
    word = word.translate(str.maketrans('', '', string.punctuation))
    return word if word in nltk_words else ''
all_words = Counter(filter(remove_punct, mr.words()))
all_words.most_common(10)

documents = [(list(filter(remove_punct, mr.words(f))), mr.categories(f)) for f in mr.fileids()]
print("Total number of documents:", len(documents))
print("Total number of words in first document:", len(documents[0][0]))


In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=[doc for doc, cat in documents])
word_vectors = model.wv
print(word_vectors['the'])
print("dimensionality word vectors:"+ str(word_vectors['the'].shape))
print("Size of vocabulary: " + str(len(word_vectors.key_to_index)))

We can find the most similar vector nearby a word using `most_similar`.

In [ ]:
word_vectors.most_similar('king')

And we can even do arithmetic with it. The most famous example of this is the `king + man - woman = queen` analogy. By adding the vector of king and man to each other, and subtracting the vector of woman, we should get the queen vector. Lets try!

In [ ]:
word_vectors.most_similar(positive=['king','woman'],negative=['man'])

We get queen as the second most similar vector. We only trained our word2vec model on our reviews dataset which is a small dataset for word2vec standards, so that makes sense.

Lastly, lets plot the data. For this, we need to represent our vectors as a 2-d space. For this, we need a dimensionality reduction technique, such as PCA or t-SNE. We use t-SNE (invented by someone who did the same master as you are doing!). It might take a while to compute the vectors below:

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
tsne = TSNE(n_components=2, random_state=0)
vectors = tsne.fit_transform(np.asarray(model.wv.vectors))
x, y = zip(*vectors)

In [ ]:
len(x), len(y)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))
plt.scatter(x, y)

t-SNE (t-distributed Stochastic Neighbor Embedding) was introduced by Laurens van der Maaten and Geoffrey Hinton.

It was published in the paper:

Laurens van der Maaten and Geoffrey Hinton. “Visualizing Data using t-SNE.” Journal of Machine Learning Research (JMLR), 2008.

Laurens van der Maaten is Dutch and did his academic training in the Netherlands:

🎓 M.Sc. in Artificial Intelligence – Maastricht University

🎓 Ph.D. in Computer Science – Tilburg University (Netherlands)

After his PhD he worked as a postdoctoral researcher at Delft University of Technology, then moved to Microsoft Research and later to Facebook AI Research (FAIR), where he became well known in the machine learning and computer vision community.

## Pretrained Word2Vec
Word2Vec actually works best when using a pretrained word vectors. This means that we would not put in data in the model to train a good representation, but we rely on external researchers that have already trained such a system on so much data the word vectors have a good representation already.

We now will use glove vectors. We can import such model like so. It might take a while to download them.

In [ ]:
import gensim.downloader
glove = gensim.downloader.load('glove-wiki-gigaword-50')

In [ ]:
glove["king"]

### Exercise 1: Limitations of basic Word Embeddings such as Word2Vec and Glove

Using all our `documents`, get the `glove` pretrained word vector for every word, take the average over all word vectors for each document.

Here is the code to do that.

In [ ]:
document_vectors = []
for doc, cat in documents:
    word_vectors_for_doc = []
    for word in doc:
        try:
            # Get the glove vector for the word
            vector = glove[word]
            word_vectors_for_doc.append(vector)
        except KeyError:
            # Handle words not in the glove vocabulary (skip or use a zero vector)
            pass  # Skipping words not in vocabulary

    if word_vectors_for_doc:
        # Calculate the average vector for the document
        avg_vector = np.mean(word_vectors_for_doc, axis=0)
        document_vectors.append((avg_vector, cat))
    else:
        # Handle empty documents after removing words not in vocabulary
        pass # Skipping empty documents

print("Number of documents with average vectors:", len(document_vectors))
# You can now use document_vectors for further analysis

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Take a few pairs of document vectors and calculate cosine similarity
# For example, comparing the first two documents
if len(document_vectors) >= 2:
    vec1 = document_vectors[0][0]
    vec2 = document_vectors[1][0]
    similarity = cosine_similarity([vec1], [vec2])[0][0]
    print(f"Cosine similarity between document 1 and document 2: {similarity}")

# You can compare other pairs as well
if len(document_vectors) >= 10 and len(document_vectors) < 2000:
    vec_pos = document_vectors[0][0] # Assuming the first document is from a positive review
    vec_neg = document_vectors[1000][0] # Assuming the 1001th document is from a negative review
    similarity_pos_neg = cosine_similarity([vec_pos], [vec_neg])[0][0]
    print(f"Cosine similarity between a positive and a negative review document: {similarity_pos_neg}")
elif len(document_vectors) >= 2000:
    vec_pos = document_vectors[0][0] # Assuming the first document is from a positive review
    vec_neg = document_vectors[1000][0] # Assuming the 1001th document is from a negative review
    similarity_pos_neg = cosine_similarity([vec_pos], [vec_neg])[0][0]
    print(f"Cosine similarity between a positive and a negative review document: {similarity_pos_neg}")
else:
    print("Not enough documents to compare positive and negative reviews.")

As you see: The cosine similarity is close to 1 for all documents because we calculated the average of all word vectors within each document. When you average many vectors, especially in longer documents, the resulting average vector tends to point in a similar general direction in the vector space, regardless of the specific nuances of the document's content. This is because the most frequent words in the dataset have a large influence on the average, making the document vectors converge towards a similar central point in the embedding space. This highlights one of the limitations of simply averaging word embeddings to represent documents.

# More Questions:

1a. Do the vectors differ a lot?

1b. When one has a number of very long documents, what will the vectors average to?

1c. Do you think this is a usefull representation of a document when using a task such as text classification?

1d. What kind of errors will you observe when a Word2Vec or Glove model for English is used on German?

1e. What will happen when certain tokens do not have an embedding in Word2Vec or Glove?  

## Bias in Word2Vec
One of the problems with Word2Vec (and with machine learning in general) is that there is lots of biases assumed by the model. Examples of biases that can be harmful when using these algorithms include gender bias and ethnicity bias. Lets check for example what happens if we take the female equivalent of `doctor`:

In [ ]:
glove.most_similar(positive=['doctor','woman'],negative=['man'])

### Exercise 2: Bias in Word2Vec

Here are other examples of bias in Glove and in Word2Vec.

In [ ]:
glove.most_similar(positive=['programmer','woman'],negative=['man'])

In [ ]:
glove.most_similar(positive=['soldier','woman'],negative=['man'])


In [ ]:
glove.most_similar(positive=['professor','woman'],negative=['man'])

## Transformers

We arrive at the state-of-the-art, Transformers models!

Several (not all) of the problems observed in Word2Vec and Glove are addressed with the word embeddings in the Transformer model and in BERT in particular. The most important one being that so-called homonyms will get different embeddings based ont he context of a word in a sentence.

Although in another course we go deeper into Transformers itself, in this section we will go through representing our dataset as vectors. We do this again with the use of a pretrained model, for example BERT. We use the `transformers` library from HuggingFace to download the model, and use it on our data. Lets install the library first and import the model.

In this section we will use Sentence Transformers library, which is a popular way of calculating embeddings for sentences using transformer models. See the documentation of the library [here](https://www.sbert.net/).
In essence, this library basically also uses BERT-based models, but uses a mean pooling algorithm to average the embeddings out over its tokens. Its also more efficient, it would take some time using BERT to compute all the embeddings for every document in our reviews dataset, Sentence Transformers is optimized to do such task.

We start with downloading the library using `pip` and importing a model.

In [ ]:
!pip install -qq sentence-transformers
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

We can encode any sentence like this:

In [ ]:
sentence_embedding = sentence_model.encode("the quick brown fox jumps over the lazy dog")
sentence_embedding.shape

We now get a vector of 384 instead of a matrix of 11 by 768. This makes it much easier to deal with.

## BERT Models
In this section, we will learn how to extract embeddings from the pre-trained BERT. Consider the sentence 'I love Maastricht'. Let's see how to obtain the contextualized word embedding of all the words in the sentence using the pre-trained BERT model with Hugging Face's transformer library.

In [ ]:
!pip install -qq transformers[torch] datasets

Lets make sure we get the same results every time we run the same code in the notebook (reproducability). We can do this by setting the seeds of the packages we use. Like this:

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Vocabulary and Tokenizers

We can tokenize a piece of text by importing the tokenizer from the `transformers` library. Then to tokenize we use the `tokenize()` function. This function splits the texts based on the items in the vocabulary.

Lets consider the `HuggingFace` library. We can check all the available pre-trained models [here](https://huggingface.co/models). For BERT, these models we can filter down on the `bert` [tag](https://huggingface.co/models?other=bert). For now, we use the [bert-base-uncased](https://huggingface.co/bert-base-uncased) model. As the name suggests, it a BERT with 12 encoders and it is trained with uncased tokens. The representation size will be 768. The `uncased` means that we have only lowercase letters in our tokenizer.

We can download and load the pretrained model like this. Lets look how the model is implemented. Can you notice the 12 layers/encoders of the model? Also notice the different type of inputs we have in the embedding layer. The word embedding, that converts the token ids we have (30522 of them) into 768. Same holds for the position embeddings and the token_type_embeddings we could put in.

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')
model

## Preprocessing the input
Define the sentence:

In [ ]:
sentence = 'I love AI'

Tokenize the sentence and obtain the tokens:

In [ ]:
tokens = tokenizer.tokenize(sentence)

Let's print the tokens:

In [ ]:
print(tokens)

**The [CLS] and [SEP] tokens:**

[CLS] (classification token):
A special token added at the beginning of every input sequence.

During pre-training, BERT learns to use the hidden state at the [CLS] position to represent the entire sequence.

For classification tasks (e.g., sentiment, NLI, topic detection), the final hidden state of [CLS] is passed to a classifier head.

[SEP] (separator token):
Marks the end of a single sequence or the boundary between two sequences.

Used in tasks with sentence pairs (e.g., question + passage, entailment).

Even for single sentences, BERT expects a [SEP] to indicate sequence end because it was trained that way.

**Why it’s needed**

Matches pre-training format
BERT was trained on inputs like:

[CLS] sentence A tokens [SEP] sentence B tokens [SEP]


If you don’t insert these tokens, the input format won’t match what the model learned, and performance drops.

Provides a fixed “sequence embedding”
The [CLS] token’s final hidden vector
ℎ
[CLS]
h
[CLS]
	​

 acts as a compact embedding for the entire sequence — the classifier head expects it.

Helps with segment information
The [SEP] token signals where one sequence ends and the next begins, enabling the model to learn relationships (e.g., in entailment tasks).

Maintains compatibility with position and segment embeddings
BERT adds token type embeddings (segment A vs B) and positional encodings; [CLS] and [SEP] help define these boundaries.

You must add [CLS] and [SEP] because BERT’s encoder and downstream heads were trained to expect them.
[CLS] provides a pooled representation for classification tasks, and [SEP] clearly signals sequence boundaries — critical for pair tasks and general consistency.

Now, we will add the `[CLS]` token at the beginning and `[SEP]` token at the end of the tokens list:

In [ ]:
tokens = ['[CLS]'] + tokens + ['[SEP]']

Let's look at our updated tokens list:

In [ ]:
print(tokens)

As we can observe, we have `[CLS]` token at the beginning and sep token at the end of our tokens list. We can also observe that length of our tokens is 5.

Say, we need to keep the length of our tokens list to 7, then, in that case, we will add two `[PAD]` tokens at the end as shown in the following:



In [ ]:
tokens = tokens + ['[PAD]'] + ['[PAD]']

Let's print our updated tokens list:

In [ ]:
print(tokens)

**Masking**

Masking is a technique used in Transformer-based models (like BERT) where some input tokens are hidden (masked) during training so the model must predict them from context. Masking is a pre-training trick where some tokens are hidden and the model learns to recover them, forcing it to understand surrounding context and build strong word and sentence representations.



As we can observe, now we have the tokens list consists of `[PAD]` tokens and the length of our tokens list is 7.

Next, we create the attention mask. The attention mask is there to let the model know which items should be taken into account when calculating the attentions. Since `[PAD]` tokens are just to pad the string and do not have any semantic meaning, we should let the model know not to take them into account.

We set the attention mask value to 1 if the token is not a `[PAD]` token else we will set the attention mask to 0 as shown below:



In [ ]:
attention_mask1 = [1 if i!= '[PAD]' else 0 for i in tokens]

Let's print the attention_mask:

In [ ]:
print(attention_mask1)


As we can observe, we have attention mask values 0 at the position where have `[PAD]` token and 1 at other positions.

Next, we convert all the tokens to their token_ids as shown below:


In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)


Let's have a look at the token_ids:

In [ ]:
print(token_ids)


From the above output, we can observe that each token is mapped to a unique token id.

Now, we convert the token_ids and attention_mask to tensors as shown below:

In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask1 = torch.tensor(attention_mask1).unsqueeze(0)


That's it. Next, we feed the token_ids and attention_mask to the pre-trained BERT model and get the embedding.

## Getting the embedding

As shown in the following code, we feed the token_ids, and attention_mask to the model and get the output of the model:

In [ ]:
print(token_ids)
print(attention_mask1)
last_hidden_state, pooler_output = model(token_ids, attention_mask = attention_mask1).to_tuple()

This output contains two keys or more keys, depending on the task at hand. In the case of `bert-base-uncased`, we get

In [ ]:
last_hidden_state.shape, pooler_output.shape

The `pooler_output` is basically the `last_hidden_state`'s `[CLS]` token through the pooling module (linear layer + activation) at the end:

In [ ]:
torch.allclose(pooler_output, model.pooler(last_hidden_state))

The size `[1,7,768]` indicates the `[batch_size, sequence_length, hidden_size]`.

Our batch size is 1, the sequence length is the token length, since we have 7 tokens, the sequence length is 7, and the hidden size is the representation (embedding) size and it is 768 for the BERT-base model.

We can obtain the representation of each token as:

- `output.last_hidden_state[0][0]` gives the representation of the first token which is `[CLS]`
- `output.last_hidden_state[0][1]` gives the representation of the second token which is 'I'
- `output.last_hidden_state[0][2]` gives the representation of the third token which is 'love'.

We can also index it with `output.last_hidden_state[:, 0, :]`. In this way, we specify that we want to have the zeroth element on the second index, we get a matrix in return with both the batch size and the embedding size (since we put a colon there).

In this way, we can obtain the contextual representation of all the tokens. This is basically the contextualized word embeddings of all the words in the given sentence.

### Exercise 3: BERT Context Sensitive Embeddings

Now we will write your own code. Put the following text (the one defined under here with three times `bank` in it) to the tokenizer, add `[CLS]` and `[SEP]` values to it and pass it through the model. Get the vector representations of each of the three tokens of `bank` in the sentence and compare them.


You can see how this differs from putting the text through a GloVe/Word2Vec model as there is now way to distinguish between different meanings for a word such as 'bank'.


Putting a string with only "bank" (add `[CLS]` and `[SEP]`) without any context through the tokenizer and model wills how you that this will result in the same vectors for 'bank'. This is in the last coding example.

In [ ]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Tokenize the sentence
tokens = tokenizer.tokenize(text)

# Add [CLS] and [SEP] tokens
tokens = ['[CLS]'] + tokens + ['[SEP]']

# Convert tokens to token IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Convert token IDs to a tensor and add a batch dimension
token_ids = torch.tensor(token_ids).unsqueeze(0)

# Get the model output
with torch.no_grad(): # Disable gradient calculation for inference
    last_hidden_state, pooler_output = model(token_ids).to_tuple()

# Find the indices of the "bank" tokens
bank_indices = [i for i, token in enumerate(tokens) if token == 'bank']

# Get the vector representations for each "bank" token
bank_vectors = [last_hidden_state[0, i, :] for i in bank_indices]

# Print the vectors (optional)
# for i, vector in enumerate(bank_vectors):
#     print(f"Vector for 'bank' instance {i+1}: {vector}")

# You can now compare these vectors (e.g., using cosine similarity)

print(text)

from sklearn.metrics.pairwise import cosine_similarity

if len(bank_vectors) >= 2:
    similarity_1_2 = cosine_similarity(bank_vectors[0].unsqueeze(0), bank_vectors[1].unsqueeze(0))[0][0]
    print(f"Cosine similarity between the first and second 'bank': {similarity_1_2}")

if len(bank_vectors) >= 3:
    similarity_1_3 = cosine_similarity(bank_vectors[0].unsqueeze(0), bank_vectors[2].unsqueeze(0))[0][0]
    print(f"Cosine similarity between the first and third 'bank': {similarity_1_3}")
    similarity_2_3 = cosine_similarity(bank_vectors[1].unsqueeze(0), bank_vectors[2].unsqueeze(0))[0][0]
    print(f"Cosine similarity between the second and third 'bank': {similarity_2_3}")

In [ ]:
# Get the vector for "bank" from the glove model
bank_vector_1 = glove["bank"]

# Access the vector for "bank" again - it will be the same
bank_vector_2 = glove["bank"]

# You can compare them to show they are identical
print(f"Are the two 'bank' vectors from Glove identical? {np.array_equal(bank_vector_1, bank_vector_2)}")

# To further illustrate, let's get vectors for "bank" in different sentences
# Note: We can't pass sentences to glove directly, only individual words.
# This limitation is what BERT overcomes. But we can show that calling glove["bank"]
# always returns the same vector.

# Even if we imagine the context, the glove vector for "bank" remains the same.
print("\nGlove vector for 'bank' (financial institution context):")
print(glove["bank"][:10]) # Print first 10 elements for brevity

print("\nGlove vector for 'bank' (river bank context):")
print(glove["bank"][:10]) # Print first 10 elements for brevity

# The vectors are the same because Glove does not consider sentence context.

In [ ]:
# Define a sentence with only "bank"
text_isolated = "bank"

# Tokenize the sentence
tokens_isolated = tokenizer.tokenize(text_isolated)

# Add [CLS] and [SEP] tokens
tokens_isolated = ['[CLS]'] + tokens_isolated + ['[SEP]']

# Convert tokens to token IDs
token_ids_isolated = tokenizer.convert_tokens_to_ids(tokens_isolated)

# Convert token IDs to a tensor and add a batch dimension
token_ids_isolated = torch.tensor(token_ids_isolated).unsqueeze(0)

# Get the model output
with torch.no_grad(): # Disable gradient calculation for inference
    last_hidden_state_isolated, pooler_output_isolated = model(token_ids_isolated).to_tuple()

# Get the vector representation for the "bank" token
# The index for "bank" is 1 because of the [CLS] token at the beginning
bank_vector_isolated = last_hidden_state_isolated[0, 1, :]

print("Vector for 'bank' in isolation:")
# print(bank_vector_isolated) # Uncomment to print the full vector

# Now compare this vector to the contextualized vectors from the previous example (Exercise 3a)
# Assuming 'bank_vectors' from Exercise 3a is still available

if 'bank_vectors' in locals() and len(bank_vectors) >= 3:
    from sklearn.metrics.pairwise import cosine_similarity

    similarity_isolated_1 = cosine_similarity(bank_vector_isolated.unsqueeze(0), bank_vectors[0].unsqueeze(0))[0][0]
    print(f"Cosine similarity between 'bank' in isolation and the first 'bank' (vault): {similarity_isolated_1}")

    similarity_isolated_2 = cosine_similarity(bank_vector_isolated.unsqueeze(0), bank_vectors[1].unsqueeze(0))[0][0]
    print(f"Cosine similarity between 'bank' in isolation and the second 'bank' (robber): {similarity_isolated_2}")

    similarity_isolated_3 = cosine_similarity(bank_vector_isolated.unsqueeze(0), bank_vectors[2].unsqueeze(0))[0][0]
    print(f"Cosine similarity between 'bank' in isolation and the third 'bank' (river): {similarity_isolated_3}")
else:
    print("Contextualized bank vectors from Exercise 3a not found. Please run Exercise 3a code first.")

# Dealing with Tokenization Problems for Chemical Formulas or RNA/DNA

In the beginning of this tutorial, we explained that there are many potenial sources of problems in Word2Vec and Glove:

- Wrong language (do not overestimate multi-language claims)
- Wrong character set (missing characters)
- Wrong tokenization leading to unknown or corrupted tokens for which no embeddings exist or that cannot be dealt with sub-words
- No relevant context for sub-word embeddings
- Missing embeddings for chemical formulas, DNA/RNA, abbreviations, etc
- Missing embeddings in general (out of vocabulairy errors) and limited sub-words
- Bias in embeddings

Language and character set detection in combination with special embeddings per language can solve the first two problems. Bias is caused by pre-training the models with text that contains various types of unwanted forms of bias.

ChemBERTa is a specialized model dealing with chemical formulas, DNABERT is a model dealing with RNA/DNA sequences. Both have specialized tokenizers, embeddings and special context embeddings derived from pre-training the models with relevant text data. BIOBERT is a good example of a BERT model specialized for Bio Medical text.

You can run the BioBERT Colab Tutorial for Medical Named Entity Recognition that can be found here: https://colab.research.google.com/github/eugenesiow/practical-ml/blob/master/notebooks/Named_Entity_Recognition_BC5CDR.ipynb



Hereunder is an exmample of a tokenization going wrong.

In [ ]:
# Example 1: A made-up word that might not be in the vocabulary
text_madeup = "This is a supercalifragilisticexpialidocious word."
tokens_madeup = tokenizer.tokenize(text_madeup)
print(f"Original text: '{text_madeup}'")
print(f"Tokens: {tokens_madeup}")
print(f"Token IDs: {tokenizer.convert_tokens_to_ids(tokens_madeup)}")
print("-" * 20)

# Example 2: A chemical formula (BERT tokenizer is not specialized for this)
text_chemical = "The formula for water is H2O."
tokens_chemical = tokenizer.tokenize(text_chemical)
print(f"Original text: '{text_chemical}'")
print(f"Tokens: {tokens_chemical}")
print(f"Token IDs: {tokenizer.convert_tokens_to_ids(tokens_chemical)}")
print("-" * 20)

# Example 3: A string with special characters or symbols not in the vocabulary
text_symbols = "Let's analyze the data: @#$!."
tokens_symbols = tokenizer.tokenize(text_symbols)
print(f"Original text: '{text_symbols}'")
print(f"Tokens: {tokens_symbols}")
print(f"Token IDs: {tokenizer.convert_tokens_to_ids(tokens_symbols)}")
print("-" * 20)

# Example 4: A word that might be split into unexpected sub-words
text_subword = "Analyzing the data is crucial."
tokens_subword = tokenizer.tokenize(text_subword)
print(f"Original text: '{text_subword}'")
print(f"Tokens: {tokens_subword}")
print(f"Token IDs: {tokenizer.convert_tokens_to_ids(tokens_subword)}")
print("-" * 20)